In [260]:
import cv2
import numpy as np


In [261]:
def rgb_to_yiq(img_rgb):
    img_yiq = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2YCrCb)
    return img_yiq

def increase_saturation(img, scale=1.5):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    s = cv2.multiply(s, scale)
    s = np.clip(s, 0, 255).astype(hsv_img.dtype)
    hsv_img = cv2.merge([h, s, v])
    img_saturated = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    return img_saturated

def increase_hue(img, scale=1.5):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    h = cv2.multiply(h, scale)
    h = np.clip(s, 0, 255).astype(hsv_img.dtype)
    hsv_img = cv2.merge([h, s, v])
    img_hued = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    return img_hued

def filter_color(img_yiq):
    y_channel, cr_channel, cb_channel = cv2.split(img_yiq)
    
    # 노란색 영역의 Cr, Cb 임계값 설정
    # Cr 빨강 청록 
    # Cb 파랑 노랑
    # 노란색 210 (Y) 146 (Cr) 16 (Cb)??
    lower_cr = 140
    upper_cr = 179
    lower_cb = 100
    upper_cb = 130
    
    # 밝기 채널의 임계값 설정 (어두운 영역 제외)
    lower_y = int(0.8 * 255)  # 최소 밝기값 (조정 가능)
    upper_y = 255  # 최대 밝기값
    
    # Cr, Cb 채널에서 임계값에 해당하는 영역 추출
    mask_cr = cv2.inRange(cr_channel, lower_cr, upper_cr)
    mask_cb = cv2.inRange(cb_channel, lower_cb, upper_cb)
    
    # Y 채널에서 임계값에 해당하는 영역 추출
    mask_y = cv2.inRange(y_channel, lower_y, upper_y)
    
    # Cr, Cb 임계값에 해당하는 마스크 생성
    color_mask_cr_cb = cv2.bitwise_and(mask_cr, mask_cb)
    
    # Y 임계값과 Cr, Cb 임계값을 결합
    color_mask = cv2.bitwise_and(color_mask_cr_cb, mask_y)
    
    return color_mask

In [262]:
def preprocess_image(frame):
    # 가우시안 블러링 적용
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    
    # 히스토그램 평활화 적용 (HSI 색상 공간에서 적용)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v_eq = cv2.equalizeHist(v)
    hsv_eq = cv2.merge([h, s, v_eq])
    equalized = cv2.cvtColor(hsv_eq, cv2.COLOR_HSV2BGR)
    
    return blurred

In [263]:
def segment_objects(img_gray):
    # Convolution을 통해 영역 확장
    kernel = np.ones((5, 5), np.float32) / 25
    convoluted_image = cv2.filter2D(img_gray, -1, kernel)
    
    # 마스크 생성
    _, thresholded = cv2.threshold(convoluted_image, 50, 255, cv2.THRESH_BINARY)
    
    # mophology
    kernel = np.ones((5, 5), np.uint8)
    morphed_img = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)
    
    return morphed_img

In [264]:
def detect_edges(img_gray):
    # Canny edge detection 적용
    edges = cv2.Canny(img_gray, 100, 200)
    
    return edges


In [265]:
def detect_quadrilaterals(edges):
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    quadrilaterals = []
    
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        if len(approx) == 4:
            quadrilaterals.append(approx)
    
    return quadrilaterals

In [266]:
def detect_circular_patterns(edges, frame):
    if edges is not None and len(edges.shape) == 2 and edges.dtype == np.uint8:
        # 원형 패턴 검출
        circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                                   param1=50, param2=30, minRadius=10, maxRadius=30)
        
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                cv2.circle(frame, (x, y), r, (0, 0, 255), 4)  # 원의 중심에 빨간색 원 그리기
    return frame


In [267]:
def detect_linear_patterns(edges, frame):
    if edges is not None and len(edges.shape) == 2 and edges.dtype == np.uint8:
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=10)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 2) 
    return frame

In [268]:
# 동영상 파일 열기
cap = cv2.VideoCapture('sample_video.mp4')

# 동영상 파일이 없으면 에러 처리
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# 출력 동영상 파일 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', 
                      fourcc, 
                      cap.get(cv2.CAP_PROP_FPS), 
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))


# 각 프레임에 대해 처리 반복
while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    # 전처리 (가우시안 필터링 및 히스토그램 평활화)
    preprocessed_frame = preprocess_image(frame)

    # H S 값 변환
    #img_hued = increase_hue(preprocessed_frame)
    #img_saturated = increase_saturation(img_yiq)

    # RGB 이미지를 YIQ 모델로 변환
    img_yiq = rgb_to_yiq(img_hued)

    # YIQ 모델에서 색상 필터링
    color_mask = filter_color(img_yiq)    

    # 외곽선 찾기
    contours, _ = cv2.findContours(color_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 원본 이미지에 외곽선 그리기
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)  # 초록색 외곽선

    cv2.imshow('Result', img_yiq)
    
    # 경계선 검출
    #edges = detect_edges(segmented_img)
    
    # 결과 프레임 출력
    #cv2.imshow('Result', frame)
    
    # 결과 동영상 파일에 프레임 추가
    out.write(frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 해제 작업
cap.release()
out.release()
cv2.destroyAllWindows()